<a href="https://colab.research.google.com/github/FabianZettl/efifapesproject/blob/main/sofifa_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import datetime
from time import sleep
import json
import os 
import re
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}


def player_scraper(url):
    list1 = []
    r  = requests.get(url,headers = headers)
    soup = bs(r.content,'html.parser')
    short_name = soup.find('h1').text.strip()
    try:
        player_id = url.split('player/')[1].split('/')[0]
    except:
        player_id = url.split('player/')[1]
    try:
        ply_name = soup.find('div',attrs = {'class':'bp3-card player'}).find('h1').text
    except:
        ply_name = ''
    try:
        nationality = soup.find('div',attrs = {'class':'meta ellipsis'}).find('a')['title']
    except:
        nationality = ''
    try:
        h_w_text = soup.find('div',attrs = {'class':'meta ellipsis'}).text
        height = h_w_text.split(' ')[-2]
        if 'cm' not in height:
            height = ''
        weight = h_w_text.split(' ')[-1]
        if 'kg' not in weight:
            weight = ''
        try:
            birth_date = h_w_text.split('(')[1].split(')')[0]
        except:
            birth_date = ''
        try:
            pref_position = ''
            for h_w in h_w_text:
                if h_w.isnumeric():
                    break
                else:
                    pref_position = pref_position + h_w
        except:
            pref_position = '' 
        try:
            age = h_w_text.split('(')[0].split(' ')[-2]
        except:
            age = ''
    except:
        height = ''
        weight = ''
        
    try:
        cards = soup.find('section',attrs = {'class':'card spacing'}).findAll('div',recursive = False)
        overall= cards[0].text.replace('Overall Rating','')
        potential = cards[1].text.replace('Potential','')
        value = cards[2].text.replace('Value','')
        wage = cards[3].text.replace('Wage','')
    except:
        overall= ''
        potential = ''
        value = ''
        wage = ''
    try:
        blocks = soup.find('div',attrs= {'class':'col col-12'}).findAll('div',recursive=False)
        profile_block = blocks[1].find('div').find('ul').findAll('li')
        pref_foot = profile_block[0].text.replace('Preferred Foot','').strip()
        week_foot = profile_block[1].text.replace('Weak Foot','').strip()
        skill_moves = profile_block[2].text.replace('Skill Moves','').strip()
        work_rate = profile_block[4].text.replace('Work Rate','').strip()
    except:
        pref_foot = ''
        week_foot = ''
        skill_moves = ''
        work_rate = ''
        
    try:
        blocks = soup.find('div',attrs= {'class':'col col-12'}).findAll('div',recursive=False)
        if len(blocks) ==6:
            country_block = blocks[-2]
            nation = country_block.find('h5').text
            nation_position = country_block.find('ul').findAll('li')[1].text.replace('Position','')
            nation_kit_num = country_block.find('ul').findAll('li')[2].text.replace('Kit Number','')
            
            
            club_block = blocks[-3]
            club = club_block.find('h5').text
            club_lis = club_block.find('ul').findAll('li')
            club_position = club_lis[1].text.replace('Position','')
            club_kit = club_lis[2].text.replace('Kit Number','')
            club_joined = club_lis[3].text.replace('Joined','')
            if 'loan' in club_joined:
                club_joined = club_joined.replace('Loaned From','') + '(Loaned)'
            club_valid_until = club_lis[4].text.replace('Contract Valid Until','')
        else:
            nation = ''
            nation_position = ''
            nation_kit_num = ''
            
            
            club_block = blocks[-2]
            club_lis = club_block.find('ul').findAll('li')
            
            club = club_block.find('h5').text
            club_position = club_lis[1].text.replace('Position','')
            club_kit = club_lis[2].text.replace('Kit Number','')
            club_joined = club_lis[3].text.replace('Joined','')
            # if 'Loan' in club_joined:
            #     club_joined = club_joined.replace('Loaned From') + '(Loaned)'
            club_valid_until = club_lis[4].text.replace('Contract Valid Until','')
    
    except:
        nation = ''
        nation_position = ''
        nation_kit_num = ''
        
        club = ''
        club_position = ''
        club_kit = ''
        club_joined = ''
        club_valid_until = ''
    try:
        blocks = soup.findAll('div',attrs= {'class':'col col-12'})[1].findAll('div',recursive = False)
        attack_block = blocks[2].find('ul').findAll('li')
        
        crossing = attack_block[0].find('span').text
        finishing = attack_block[1].find('span').text
        heading_acc = attack_block[2].find('span').text
        short_pass = attack_block[3].find('span').text
        volleys = attack_block[4].find('span').text
        #-----------------------------------------------
        skill_block = blocks[3].find('ul').findAll('li')
        
        dribbling = skill_block[0].find('span').text
        curve = skill_block[1].find('span').text
        fk_accuracy = skill_block[2].find('span').text
        long_pass = skill_block[3].find('span').text
        ball_control = skill_block[4].find('span').text
        #-----------------------------------------------
        movement_block = blocks[4].find('ul').findAll('li')
        
        acceleration = movement_block[0].find('span').text
        sprint_speed = movement_block[1].find('span').text
        agility = movement_block[2].find('span').text
        reactions = movement_block[3].find('span').text
        balance = movement_block[4].find('span').text
        #-----------------------------------------------
        power_block = blocks[5].find('ul').findAll('li')

        shot_power = power_block[0].find('span').text
        jumping = power_block[1].find('span').text
        stamina = power_block[2].find('span').text
        strength = power_block[3].find('span').text
        long_shot = power_block[4].find('span').text
        #-----------------------------------------------
        mentality_block = blocks[6].find('ul').findAll('li')
        
        aggresions = mentality_block[0].find('span').text
        interceptions = mentality_block[1].find('span').text
        positioning = mentality_block[2].find('span').text
        vision = mentality_block[3].find('span').text
        penalties = mentality_block[4].find('span').text
        composure = mentality_block[5].find('span').text
        #-----------------------------------------------
        defending_block = blocks[7].find('ul').findAll('li')
        
        def_awareness = defending_block[0].find('span').text
        standing_tackle = defending_block[1].find('span').text
        sliding_tackle = defending_block[2].find('span').text
  
        #-----------------------------------------------
        goal_block = blocks[8].find('ul').findAll('li')
        
        gk_diving = goal_block[0].find('span').text
        gk_handling = goal_block[1].find('span').text
        gk_kicking = goal_block[2].find('span').text
        gk_position = goal_block[3].find('span').text
        gk_reflex = goal_block[4].find('span').text
        #--------traits---------------------------------------
        try:
            trait_block = blocks[9].find('ul').findAll('li')
            
            traits = ''
            for tra in trait_block:
                traits = traits +','+ tra.text
            if traits[-1] == ',':
                traits = traits[:-1]
            if traits[0] == ',':
                traits = traits[1:]
        except:
            traits = ''
    except:
        crossing = ''
        finishing = ''
        heading_acc = ''
        short_pass = ''
        volleys = ''
        dribbling = ''
        curve = ''
        fk_accuracy = ''
        long_pass = ''
        ball_control = ''
        acceleration = ''
        sprint_speed = ''
        agility = ''
        reactions = ''
        balance = ''
        aggresions = ''
        interceptions = ''
        positioning = ''
        vision = ''
        penalties = ''
        composure = ''
        def_awareness = ''
        standing_tackle = ''
        sliding_tackle = ''
        gk_diving = ''
        gk_handling = ''
        gk_kicking = ''
        gk_position = ''
        gk_reflex = ''
        traits = ''
        shot_power = ''
        jumping = ''
        stamina = ''
        strength = ''
        long_shot = ''
    datan = {
        'Profile URL': url,
        'Player ID': player_id,
        'Player Name': ply_name,
        'Player Short Name': short_name,
        'Nationality':nationality,
        'Overall': overall,
        'Potential': potential,
        'Height': height,
        'Weight': weight,
        'PreferredFoot': pref_foot,
        'BirthDate': birth_date,
        'Age': age,
        'PreferredPositions': pref_position,
        'PlayerWorkRate': work_rate,
        'WeekFoot': week_foot,
        'SkillMoves': skill_moves,
        'Value': value,
        'Wage': wage,
        'Nation': nation,
        'Nation_Position': nation_position,
        'NationKitNumber': nation_kit_num,
        'Club': club,
        'Club_Position': club_position,
        'Club_KitNumber': club_kit,
        'Club_Joined': club_joined,
        'Club_ContractLength': club_valid_until,
        'Crossing': crossing,
        'Finishing': finishing,
        'Heading Accuracy': heading_acc,
        'Short Passing': short_pass,
        'Volleys': volleys,
        'Dribbling': dribbling,
        'Curve': curve,
        'FK Accuracy': fk_accuracy,
        'Long Passing': long_pass,
        'Ball Control': ball_control,
        'Acceleration': acceleration,
        'Sprint Speed': sprint_speed,
        'Agility': agility,
        'Reactions': reactions,
        'Balance': balance,
        'Shot Power': shot_power,
        'Jumping': jumping,
        'Stamina': stamina,
        'Strength':strength,
        'Long Shot': long_shot,
        'Aggression': aggresions,
        'Interceptions': interceptions,
        'Positioning': positioning,
        'Vision': vision,
        'Penalties': penalties,
        'Composure': composure,
        'Defensive Awareness': def_awareness,
        'Standing Tackle': standing_tackle,
        'Sliding Tackle': sliding_tackle,
        'GK Diving': gk_diving,
        'GK Handling': gk_handling,
        'GK Kicking': gk_kicking,
        'GK Positioning': gk_position,
        'GK Reflexes': gk_reflex,
        'Traits' :traits
    }
    list1.append(datan)
    
    return list1
    
# https://sofifa.com/players?type=updated

def main():
    print('1. Homepage Scrape')
    print('2. Team Scrape')
    print('3. Player Scrape')
    print('4. "Updated" tab Scrape')
    print('5. Scrape Filtered URLs')
    print('6. Exit Code')
    choice = int(input('Enter Choice(1-6): '))
    if choice==3:
        url = input('Enter Player Url: ')
        listan = player_scraper(url)

        try:
            player_id = url.split('player/')[1].split('/')[0]
        except:
            player_id = url.split('player/')[1]
        df = pd.DataFrame(listan).to_excel(f'player_id_{player_id}.xlsx',encoding = 'utf-8-sig',index = False)
        from google.colab import files
        files.download(f'player_id_{player_id}.xlsx')
    elif choice == 1:
        player_urls = []
        listan = []
        count = 60
        while True:
            if count == 20060:
                break
            prev_len = len(player_urls)
            try:
                url = f'https://sofifa.com/?offset={count}'
                print(f'Scraping Players URL from Homepage: {url}')
                r  = requests.get(url,headers = headers)
                soup = bs(r.content,'html.parser')

                pr = soup.find('table',attrs = {'class':'table table-hover persist-area'}).findAll('tr')[1:]
                for p in pr:
                    try:
                        player_url = 'https://sofifa.com' +  str(p.findAll('td')[1].find('a')['href'])
                        if player_url in player_urls:
                            pass
                        else:
                            player_urls.append(player_url)
                    except:
                        pass
            
            except:
                pass
            if len(player_urls) == prev_len:
                break
            count = count + 60
        sleep(2)
        print(f'\nTotal Players Profile found from homepage: {len(player_urls)}\n')
        sleep(3)
        print('Scraping Starts:')

        progress = 1
        totals = len(player_urls)
        for url in player_urls:
            print(f'[{progress}/{totals}] Scraping Player URL: {url}')
            list2 = player_scraper(url)
            listan = listan + list2
            progress = progress + 1
        
        df = pd.DataFrame(listan).to_excel(f'players_from_homepage.xlsx',encoding = 'utf-8-sig',index = False)
        from google.colab import files
        files.download(f'players_from_homepage.xlsx')
    #---adding----------
    elif choice == 4:
        player_urls = []
        listan = []
        count = 60
        while True:
            if count == 20060:
                break
            prev_len = len(player_urls)
            try:
                url = f'https://sofifa.com/players?type=updated&offset={count}'
                print(f'Scraping Players URL from Update Tab: {url}')
                r  = requests.get(url,headers = headers)
                soup = bs(r.content,'html.parser')

                pr = soup.find('table',attrs = {'class':'table table-hover persist-area'}).findAll('tr')[1:]
                for p in pr:
                    try:
                        player_url = 'https://sofifa.com' +  str(p.findAll('td')[1].find('a')['href'])
                        if player_url in player_urls:
                            pass
                        else:
                            player_urls.append(player_url)
                    except:
                        pass
            except:
                pass
            if len(player_urls) == prev_len:
                break
            count = count + 60
        sleep(2)
        print(f'\nTotal Players Profile found from Update Tab: {len(player_urls)}\n')
        sleep(3)
        print('Scraping Starts:')

        progress = 1
        totals = len(player_urls)
        for url in player_urls:
            print(f'[{progress}/{totals}] Scraping Player URL: {url}')
            list2 = player_scraper(url)
            listan = listan + list2
            progress = progress + 1
        
        df = pd.DataFrame(listan).to_excel(f'players_from_update_tab.xlsx',encoding = 'utf-8-sig',index = False)
        from google.colab import files
        files.download(f'players_from_update_tab.xlsx')
    elif choice == 5:
        filter_url = input('Enter Filtered URL: ')
        player_urls = []
        listan = []
        count = 60
        while True:
            if count == 20060:
                break
            prev_len = len(player_urls)
            try:
                url = f'{filter_url}&offset={count}'
                print(f'Scraping Filtered Players URL: {url}')
                r  = requests.get(url,headers = headers)
                soup = bs(r.content,'html.parser')

                pr = soup.find('table',attrs = {'class':'table table-hover persist-area'}).findAll('tr')[1:]
                for p in pr:
                    try:
                        player_url = 'https://sofifa.com' +  str(p.findAll('td')[1].find('a')['href'])
                        if player_url in player_urls:
                            pass
                        else:
                            player_urls.append(player_url)
                    except:
                        pass
            except:
                pass
            if len(player_urls) == prev_len:
                break
            count = count + 60
        sleep(2)
        print(f'\nTotal Filtered Players Profile found: {len(player_urls)}\n')
        sleep(3)
        print('Scraping Starts:')

        progress = 1
        totals = len(player_urls)
        for url in player_urls:
            print(f'[{progress}/{totals}] Scraping Player URL: {url}')
            list2 = player_scraper(url)
            listan = listan + list2
            progress = progress + 1
        
        df = pd.DataFrame(listan).to_excel(f'players_from_filtered_url.xlsx',encoding = 'utf-8-sig',index = False)
        from google.colab import files
        files.download(f'players_from_filtered_url.xlsx')
    #----adding-done---------------------------------

    elif choice == 2:
        team_url = input('Enter Team Url(ex. https://sofifa.com/team/1/arsenal):  ')
        listan = []
        r  = requests.get(team_url,headers = headers)
        soup = bs(r.content,'html.parser')
        player_block = soup.findAll('table',attrs = {'class':'table table-hover persist-area'})
        player_urls = []
        squads_trs = player_block[0].findAll('tr')
        for tr in squads_trs:
            try:
                player_url = 'https://sofifa.com' + str(tr.findAll('td')[1].find('a')['href'])
                player_urls.append(player_url)
            except:
                pass
        loan_trs = player_block[1].findAll('tr')
        for tr in loan_trs:
            try:
                player_url = 'https://sofifa.com' + str(tr.findAll('td')[1].find('a')['href'])
                player_urls.append(player_url)
            except:
                pass
        progress = 1
        totals = len(player_urls)
        for url in player_urls:
            print(f'[{progress}/{totals}] Scraping Player URL: {url}')
            list2 = player_scraper(url)
            listan = listan + list2
            progress = progress + 1
        
        team_name = team_url.split('/')[-1]
        df = pd.DataFrame(listan).to_excel(f'team_name_{team_name}.xlsx',encoding = 'utf-8-sig',index = False)
        from google.colab import files
        files.download(f'team_name_{team_name}.xlsx')
    else:
        exit()
main()

1. Homepage Scrape
2. Team Scrape
3. Player Scrape
4. "Updated" tab Scrape
5. Scrape Filtered URLs
6. Exit Code
